# Сверточная нейросеть для решения задачи классификации картинок

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from torchvision import datasets, transforms

data_tfs = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5), (0.5))
])

train_data = datasets.CIFAR10(root='/kaggle/working/cifar10_data', train=True, download=True, transform=data_tfs)
test_data = datasets.CIFAR10(root='/kaggle/working/cifar10_data', train=False, download=True, transform=data_tfs)

train_data

In [ ]:
train_size = int(len(train_data) * 0.8)
val_size = len(train_data) - train_size

train_data, val_data = torch.utils.data.random_split(train_data, [train_size, val_size])

In [ ]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True, pin_memory=True)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=64, shuffle=False, pin_memory=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=64, shuffle=False, pin_memory=True)

In [ ]:
images, labels = next(iter(train_loader))
images.shape, labels.shape

In [ ]:
def show_images(images, labels):
    f, axes = plt.subplots(1, 10, figsize=(30, 5))
    
    for i, axis in enumerate(axes):
        img = images[i].numpy()
        img = np.transpose(img, (1, 2, 0))
        
        axes[i].imshow(img)
        axes[i].set_title(labels[i].numpy())
        
    plt.show()

In [ ]:
show_images(images, labels)

In [ ]:
# моудль где определены слои для нейронной сети
import torch.nn as nn
# модуль где определены функции активации
import torch.nn.functional as F
from sklearn.metrics import accuracy_score

In [ ]:
class ConvNet(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=(3, 3), padding=1) #32x32
        self.pool1 = nn.MaxPool2d(kernel_size=(2,2)) #16x16
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(3, 3), padding=1) #16x16
        self.pool2 = nn.MaxPool2d(kernel_size=(2,2)) #8x8
        self.flatten = nn.Flatten()
        
        self.fc1 = nn.Linear(8 * 8 * 32, 1024)
        self.fc2 = nn.Linear(1024, 1024)
        self.fc3 = nn.Linear(1024, 10)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        
        x = self.flatten(x)
        
        x = F.elu(self.fc1(x))
        x = F.elu(self.fc2(x))
        x = self.fc3(x)
        
        return x

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
conv_net = ConvNet()
conv_net = conv_net.to(device)

In [ ]:
loss_fn = torch.nn.CrossEntropyLoss()
learning_rate = 1e-3
optimizer = torch.optim.Adam(conv_net.parameters(), lr=learning_rate)

In [ ]:
def evaluate(model, dataloader, loss_fn):
    losses = []
    num_correct = 0
    num_elements = len(dataloader)
    
    for i, batch in enumerate(dataloader):
        X_batch, y_batch = batch
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)
        
        with torch.no_grad():
            logits = model(X_batch)
            
            loss = loss_fn(logits, y_batch)
            losses.append(loss.item())
            
            y_pred = torch.argmax(logits, dim=1)
            num_correct += torch.sum(y_pred == y_batch)
    
    accuracy = num_correct / num_elements
    
    return accuracy, np.mean(losses)

In [ ]:
from IPython.display import clear_output

def train(model, loss_fn, optimizer, n_epoch=3):
    train_losses = []
    train_accuracies = []
    
    val_losses = []
    val_accuracies = []
    
    for epoch in range(n_epoch):
        model.train(True)
        
        for i, batch in enumerate(train_loader):
            X_batch, y_batch = batch
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)
            
            logits = model(X_batch)
            
            loss = loss_fn(logits, y_batch)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            
            if i % 50 == 0:
                fig, axes = plt.subplots(2, 2, figsize=(14, 10))
                train_losses.append(loss.item())
                
                model_answers = torch.argmax(logits, dim=1)
                train_accuracy = accuracy_score(y_batch.numpy(), model_answers)
                train_accuracies.append(train_accuracy)
                
                train_iterations = np.array(range(len(train_losses)))*50
                
                axes[0, 0].plot(train_iterations, train_losses)
                axes[0, 0].set_title('Train losses')
                axes[0, 0].set(xlabel='Iterations', ylabel='Loss')
                
                axes[0, 1].plot(train_iterations, train_accuracies)
                axes[0, 1].set_title('Train accuracies')
                axes[0, 1].set(xlabel='Iterations', ylabel='Accuracy')
                
                val_iterations = np.array(range(len(val_losses)))*50
                
                axes[1, 0].plot(val_iterations, val_losses)
                axes[1, 0].set_title('Val losses')
                axes[1, 0].set(xlabel='Iterations', ylabel='Loss')
                
                axes[1, 1].plot(val_iterations, val_accuracies)
                axes[1, 1].set_title('Val accuracies')
                axes[1, 1].set(xlabel='Iterations', ylabel='Accuracy')
                
                plt.show()
                
                clear_output(wait=True)
            
        model.train(False)
        
        val_accuracy, val_loss = evaluate(model, val_loader, loss_fn=loss_fn)
        val_losses.append(val_loss)
        val_accuracies.append(val_accuracy)
        
    return model, train_losses, val_losses, val_accuracies

In [ ]:
conv_net, train_losses_conv, val_losses_conv, val_accuracies_conv = train(conv_net, loss_fn, optimizer, n_epoch=8)

In [ ]:
plt.figure(figsize=(10, 6))
plt.title('Train losses')
plt.plot(np.arange(len(train_losses_conv)), train_losses_conv, label='train losses')
plt.plot(np.arange(1, len(val_losses_conv) + 1) * (len(train_losses_conv) // len(val_losses_conv)), val_losses_conv, label='val losses')
plt.scatter(np.arange(1, len(val_losses_conv) + 1) * (len(train_losses_conv) // len(val_losses_conv)), val_losses_conv, label='val losses')
plt.xlabel('number of iterations')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
plt.title('Val accuracies')
plt.plot(np.arange(len(val_accuracies_conv)), val_accuracies_conv, label='val accuracies')
plt.xlabel('number of iterations')
plt.legend()
plt.show()

In [ ]:
train_accuracy, _ = evaluate(conv_net, train_loader, loss_fn)
print(f'Train accuracy: {train_accuracy}')

test_accuracy, _ = evaluate(conv_net, test_loader, loss_fn)
print(f'Test accuracy: {test_accuracy}')